# Chapter 70: Building Dashboards

## Learning Objectives

By the end of this chapter, you will be able to:

- Understand the principles of effective dashboard design for monitoring and interacting with prediction systems
- Choose the right dashboarding tool based on requirements (interactivity, deployment, audience)
- Build interactive dashboards using Streamlit to visualise NEPSE predictions and model performance
- Create more complex, multi‑page dashboards with Plotly Dash
- Use Panel for building dashboards that work seamlessly in Jupyter notebooks and as standalone apps
- Deploy dashboards using cloud services (Streamlit Sharing, Heroku, AWS) or within your organisation
- Implement real‑time updates using WebSockets or polling
- Design dashboards for different audiences: data scientists, traders, and management
- Apply best practices for layout, colour, and user experience

---

## Introduction

A machine learning model is only useful if its outputs can be understood and acted upon. While APIs serve predictions to other systems, dashboards provide a human‑friendly interface to monitor, explore, and interact with the model. For the NEPSE prediction system, a dashboard could show:

- Real‑time predictions for major stocks
- Historical performance (actual vs. predicted)
- Feature importance and drift metrics
- Alerts for anomalies or model degradation

Dashboards serve different audiences: traders need fast, actionable signals; data scientists need deep dives into model behaviour; management needs high‑level summaries. Building effective dashboards requires a blend of technical skills (connecting to data sources, creating visualisations) and design thinking (layout, colour, simplicity).

In this chapter, we will explore several Python‑based dashboarding libraries and build example dashboards for the NEPSE system. We'll start with **Streamlit** for rapid prototyping, then move to **Dash** for more complex, interactive applications, and touch on **Panel** for Jupyter‑integrated workflows. We'll also discuss deployment options to make your dashboards accessible to stakeholders.

---

## 70.1 Dashboard Design Principles

Before writing any code, consider the purpose and audience of your dashboard. A well‑designed dashboard follows these principles:

### 70.1.1 Know Your Audience

- **Traders**: Need real‑time data, alerts, and simple visualisations (e.g., price charts with predictions). They want to make quick decisions.
- **Data Scientists**: Need detailed metrics (accuracy, drift), feature distributions, and the ability to drill down. They want to diagnose model issues.
- **Management**: Need high‑level summaries (overall accuracy, cost per prediction) and trends over time. They want to assess business impact.

### 70.1.2 Focus on Key Metrics

Don't try to show everything. Identify the most important 3‑5 metrics and make them prominent. Secondary information can be in expandable sections or tabs.

### 70.1.3 Use Appropriate Visualisations

- **Time series**: Line charts.
- **Distributions**: Histograms, box plots.
- **Comparisons**: Bar charts.
- **Relationships**: Scatter plots.
- **Proportions**: Pie charts (use sparingly).

### 70.1.4 Layout and Hierarchy

- Place the most important information at the top (above the fold).
- Group related information together.
- Use white space to avoid clutter.
- Ensure the dashboard is readable on different screen sizes.

### 70.1.5 Colour and Accessibility

- Use colour consistently (e.g., red for alerts, green for positive).
- Avoid too many colours.
- Consider colour‑blind friendly palettes.
- Ensure sufficient contrast for text.

### 70.1.6 Interactivity

- Allow users to filter by date, symbol, etc.
- Provide tooltips on hover for more detail.
- Enable drilling down (clicking on a chart to see more details).

---

## 70.2 Dashboard Architecture

A dashboard typically has three layers:

1. **Data layer**: Sources of data (databases, APIs, data warehouses). For the NEPSE system, this could be a PostgreSQL database with predictions, a Redis cache for real‑time values, or flat files.
2. **Backend layer**: A server that fetches data, performs any necessary aggregations, and serves it to the frontend. For simple dashboards, the frontend may query the data source directly.
3. **Frontend layer**: The dashboard application itself, running in the browser.

For Python dashboards, the frontend and backend are often combined in a single application that runs on a server and renders HTML/JavaScript in the browser.

![Dashboard Architecture](images/dashboard_arch.png)

---

## 70.3 Streamlit

Streamlit is a Python library that turns data scripts into interactive web apps with minimal code. It is ideal for rapid prototyping and internal tools.

### 70.3.1 Installation

```bash
pip install streamlit pandas numpy plotly
```

### 70.3.2 Basic Streamlit App

Create a file `nepse_dashboard.py`:

```python
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

# Set page configuration
st.set_page_config(
    page_title="NEPSE Prediction Dashboard",
    page_icon="📈",
    layout="wide"
)

# Title
st.title("📈 NEPSE Stock Prediction Dashboard")
st.markdown("Real‑time predictions and model monitoring for the Nepal Stock Exchange")

# Sidebar for controls
st.sidebar.header("Controls")
symbol = st.sidebar.selectbox(
    "Select Stock Symbol",
    ["NABIL", "NTC", "SBI", "HRL", "NICA"]
)
days = st.sidebar.slider("Days to display", 7, 90, 30)

# Simulate loading data (in reality, you'd query a database)
@st.cache_data
def load_data(symbol, days):
    # Simulate historical data
    dates = pd.date_range(end=datetime.now(), periods=days)
    np.random.seed(42)  # for reproducibility
    actual = 1000 + np.cumsum(np.random.randn(days) * 10)
    predicted = actual + np.random.randn(days) * 5
    return pd.DataFrame({
        'date': dates,
        'actual': actual,
        'predicted': predicted
    })

df = load_data(symbol, days)

# Display metrics
col1, col2, col3, col4 = st.columns(4)
with col1:
    latest_actual = df['actual'].iloc[-1]
    st.metric("Latest Actual", f"{latest_actual:.2f}")
with col2:
    latest_pred = df['predicted'].iloc[-1]
    st.metric("Latest Prediction", f"{latest_pred:.2f}")
with col3:
    error = latest_pred - latest_actual
    st.metric("Error", f"{error:.2f}")
with col4:
    mae = (df['actual'] - df['predicted']).abs().mean()
    st.metric("MAE (30d)", f"{mae:.2f}")

# Plot
fig = px.line(df, x='date', y=['actual', 'predicted'],
              title=f"{symbol} - Actual vs Predicted")
st.plotly_chart(fig, use_container_width=True)

# Show data table
with st.expander("Show raw data"):
    st.dataframe(df)

# Add a section for model metrics
st.header("Model Performance Metrics")
col1, col2 = st.columns(2)
with col1:
    # Distribution of errors
    df['error'] = df['predicted'] - df['actual']
    fig2 = px.histogram(df, x='error', nbins=30, title="Error Distribution")
    st.plotly_chart(fig2, use_container_width=True)
with col2:
    # Feature importance (placeholder)
    features = ['SMA_20', 'RSI', 'Volume', 'Lag1', 'Lag2']
    importance = [0.25, 0.20, 0.15, 0.30, 0.10]
    fig3 = px.bar(x=features, y=importance, title="Feature Importance")
    st.plotly_chart(fig3, use_container_width=True)
```

**Explanation:**  
- We set up the page with a title and sidebar controls.
- `@st.cache_data` caches the data loading function, so it only runs when the symbol or days change.
- Columns are used to display metrics in a row.
- Plotly Express creates interactive line and histogram charts.
- An expandable section shows the raw data table.

### 70.3.3 Running the App

```bash
streamlit run nepse_dashboard.py
```

Streamlit opens a browser window with the app. Any code changes are automatically reflected (hot reloading).

### 70.3.4 Adding Real‑Time Updates

For real‑time data, you can use `st.empty()` and update it in a loop, or use WebSockets. However, Streamlit is not designed for sub‑second updates; for true real‑time, consider Dash or a dedicated frontend.

**Example of periodic refresh:**

```python
placeholder = st.empty()
while True:
    # Fetch new data
    new_data = fetch_latest()
    with placeholder.container():
        # Redraw charts
        st.line_chart(new_data)
    time.sleep(60)  # update every minute
    st.rerun()
```

**Note:** `st.rerun()` is experimental; a better approach is to use Streamlit's built‑in `@st.cache_data` with a TTL and let the user refresh manually.

### 70.3.5 Deploying Streamlit Apps

Streamlit offers **Streamlit Community Cloud** for free deployment (public apps). You can also deploy on your own infrastructure:

- **Docker**: Package the app in a Docker container.
- **Heroku**: Use a `Procfile` with `web: streamlit run app.py --server.port $PORT`.
- **AWS EC2**: Run as a service with systemd.
- **Kubernetes**: Deploy as a container.

**Example Dockerfile:**

```dockerfile
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

EXPOSE 8501

CMD ["streamlit", "run", "nepse_dashboard.py", "--server.port=8501", "--server.address=0.0.0.0"]
```

---

## 70.4 Dash by Plotly

Dash is a more powerful framework for building analytical web applications. It uses Flask for the backend and React for the frontend, and is highly customisable. Dash apps are composed of two parts: the **layout** (HTML components) and **callbacks** (interactivity).

### 70.4.1 Installation

```bash
pip install dash plotly pandas
```

### 70.4.2 Basic Dash App

Create a file `app.py`:

```python
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Initialize the app
app = dash.Dash(__name__)
server = app.server  # for deployment

# Sample data function (same as before)
def load_data(symbol, days):
    dates = pd.date_range(end=datetime.now(), periods=days)
    np.random.seed(42)
    actual = 1000 + np.cumsum(np.random.randn(days) * 10)
    predicted = actual + np.random.randn(days) * 5
    return pd.DataFrame({
        'date': dates,
        'actual': actual,
        'predicted': predicted
    })

# Define the layout
app.layout = html.Div([
    html.H1("📈 NEPSE Stock Prediction Dashboard", style={'text-align': 'center'}),

    html.Div([
        html.Label("Select Stock Symbol"),
        dcc.Dropdown(
            id='symbol-dropdown',
            options=[
                {'label': 'NABIL', 'value': 'NABIL'},
                {'label': 'NTC', 'value': 'NTC'},
                {'label': 'SBI', 'value': 'SBI'},
                {'label': 'HRL', 'value': 'HRL'},
                {'label': 'NICA', 'value': 'NICA'}
            ],
            value='NABIL'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '20px'}),

    html.Div([
        html.Label("Days to display"),
        dcc.Slider(
            id='days-slider',
            min=7,
            max=90,
            step=1,
            value=30,
            marks={i: str(i) for i in [7, 30, 60, 90]}
        ),
    ], style={'width': '50%', 'display': 'inline-block', 'padding': '20px'}),

    dcc.Graph(id='price-chart'),

    html.Div([
        dcc.Graph(id='error-dist', style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(id='feature-importance', style={'width': '48%', 'display': 'inline-block'})
    ]),

    html.Div(id='metrics-display', style={'padding': '20px', 'font-size': '20px'})
])

# Define callbacks
@app.callback(
    [Output('price-chart', 'figure'),
     Output('error-dist', 'figure'),
     Output('feature-importance', 'figure'),
     Output('metrics-display', 'children')],
    [Input('symbol-dropdown', 'value'),
     Input('days-slider', 'value')]
)
def update_charts(symbol, days):
    df = load_data(symbol, days)

    # Price chart
    fig1 = px.line(df, x='date', y=['actual', 'predicted'],
                   title=f"{symbol} - Actual vs Predicted")

    # Error distribution
    df['error'] = df['predicted'] - df['actual']
    fig2 = px.histogram(df, x='error', nbins=30, title="Error Distribution")

    # Feature importance (placeholder)
    features = ['SMA_20', 'RSI', 'Volume', 'Lag1', 'Lag2']
    importance = [0.25, 0.20, 0.15, 0.30, 0.10]
    fig3 = px.bar(x=features, y=importance, title="Feature Importance")

    # Metrics
    latest_actual = df['actual'].iloc[-1]
    latest_pred = df['predicted'].iloc[-1]
    mae = df['error'].abs().mean()
    metrics_text = f"Latest Actual: {latest_actual:.2f} | Latest Prediction: {latest_pred:.2f} | MAE (30d): {mae:.2f}"

    return fig1, fig2, fig3, metrics_text

if __name__ == '__main__':
    app.run_server(debug=True)
```

**Explanation:**  
- The layout uses HTML components from `dash.html` and interactive components from `dash.dcc`.
- A single callback updates all charts and metrics whenever the dropdown or slider changes.
- The callback returns multiple outputs, which Dash handles efficiently.

### 70.4.3 Running the App

```bash
python app.py
```

The app runs on `http://localhost:8050`.

### 70.4.4 Adding Real‑Time Updates with WebSockets

Dash supports real‑time updates via **Dash Core Components** like `dcc.Interval`. This component triggers a callback at regular intervals.

**Example: Add an interval to refresh data every 10 seconds**

```python
app.layout = html.Div([
    dcc.Interval(id='interval-component', interval=10*1000, n_intervals=0),
    # ... other components ...
])

@app.callback(
    Output('price-chart', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('symbol-dropdown', 'value'),
     Input('days-slider', 'value')]
)
def update_charts(n, symbol, days):
    # Fetch fresh data
    df = load_data(symbol, days)  # in reality, get new data
    fig = px.line(...)
    return fig
```

**Explanation:**  
The `dcc.Interval` increments `n_intervals` every 10 seconds, triggering the callback to update the chart with new data.

### 70.4.5 Deploying Dash Apps

Dash apps are Flask apps and can be deployed similarly:

- **Heroku**: Use a `Procfile` with `web: gunicorn app:server`.
- **AWS Elastic Beanstalk**: Deploy as a Python application.
- **Docker**: Containerise and run on any platform.
- **Dash Enterprise**: Plotly's commercial platform for enterprise deployment.

---

## 70.5 Panel

Panel is a library for creating dashboards that work seamlessly in Jupyter notebooks and as standalone apps. It integrates with many plotting libraries (Bokeh, Matplotlib, Plotly) and supports both reactive and callback‑based programming.

### 70.5.1 Installation

```bash
pip install panel pandas numpy plotly
```

### 70.5.2 Basic Panel App

Create a file `nepse_panel.py`:

```python
import panel as pn
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime

pn.extension('plotly')  # enable Plotly support

# Load data function
def load_data(symbol, days):
    dates = pd.date_range(end=datetime.now(), periods=days)
    np.random.seed(42)
    actual = 1000 + np.cumsum(np.random.randn(days) * 10)
    predicted = actual + np.random.randn(days) * 5
    return pd.DataFrame({
        'date': dates,
        'actual': actual,
        'predicted': predicted
    })

# Define widgets
symbol_widget = pn.widgets.Select(name='Stock Symbol', options=['NABIL', 'NTC', 'SBI', 'HRL', 'NICA'])
days_widget = pn.widgets.IntSlider(name='Days', start=7, end=90, step=1, value=30)

# Reactive function that updates when widgets change
@pn.depends(symbol_widget, days_widget)
def create_plots(symbol, days):
    df = load_data(symbol, days)

    # Price chart
    fig1 = px.line(df, x='date', y=['actual', 'predicted'],
                   title=f"{symbol} - Actual vs Predicted")

    # Error distribution
    df['error'] = df['predicted'] - df['actual']
    fig2 = px.histogram(df, x='error', nbins=30, title="Error Distribution")

    # Layout
    return pn.Column(
        pn.Row(fig1),
        pn.Row(fig2),
        pn.Row(f"Latest Actual: {df['actual'].iloc[-1]:.2f} | "
               f"Latest Prediction: {df['predicted'].iloc[-1]:.2f} | "
               f"MAE: {df['error'].abs().mean():.2f}")
    )

# Main layout
layout = pn.Column(
    pn.Row(symbol_widget, days_widget),
    create_plots
)

layout.servable()
```

### 70.5.3 Running Panel Apps

```bash
panel serve nepse_panel.py
```

This starts a Bokeh server on `http://localhost:5006/nepse_panel`.

### 70.5.4 Panel in Jupyter

Panel apps can also be displayed directly in Jupyter notebooks by adding `layout` at the end of a cell. This makes it great for exploratory work.

### 70.5.5 Deploying Panel

Panel apps can be deployed using:

- **Bokeh server**: As above.
- **Docker**: Package the app.
- **Heroku**: Use a `Procfile` with `web: panel serve nepse_panel.py --address 0.0.0.0 --port $PORT --allow-websocket-origin=*`.

---

## 70.6 Choosing the Right Tool

| Tool       | Best For                                      | Learning Curve | Interactivity | Deployment |
|------------|-----------------------------------------------|----------------|---------------|------------|
| Streamlit  | Rapid prototyping, internal tools             | Low            | Good          | Easy       |
| Dash       | Complex, multi‑page apps, production          | Medium         | Excellent     | Flexible   |
| Panel      | Jupyter‑integrated workflows, research        | Low to Medium  | Good          | Good       |
| Voilà      | Turn Jupyter notebooks into dashboards        | Low            | Limited       | Easy       |

For the NEPSE system, Streamlit is great for a quick internal dashboard. If you need more sophisticated interactivity (e.g., cross‑filtering, custom JavaScript), Dash is the way to go. Panel is excellent if your team already works heavily in Jupyter.

---

## 70.7 Best Practices for Dashboard Deployment

1. **Secure your dashboard**: If the dashboard contains sensitive information, add authentication. Streamlit Community Cloud offers password protection; Dash can integrate with Flask‑Login.
2. **Optimise performance**: Use caching (e.g., `@st.cache_data` in Streamlit, `functools.lru_cache` in Dash) to avoid repeated expensive queries.
3. **Monitor dashboard usage**: Track which features are used most; this can guide future development.
4. **Plan for mobile**: Consider a separate mobile‑optimised view or ensure your layout is responsive.
5. **Document the dashboard**: Provide a short guide for users explaining what each chart shows and how to use filters.
6. **Keep it simple**: Resist the urge to add too many features. A clean, focused dashboard is more valuable.

---

## Chapter Summary

In this chapter, we explored building dashboards to visualise and interact with the NEPSE prediction system. We covered:

- Dashboard design principles: know your audience, focus on key metrics, use appropriate visualisations.
- Streamlit for rapid development, with a complete example dashboard.
- Dash for more complex, production‑grade applications, with callbacks and real‑time updates.
- Panel for Jupyter‑integrated workflows.
- How to deploy dashboards on various platforms.
- Choosing the right tool based on requirements.
- Best practices for dashboard deployment and maintenance.

With a dashboard, the NEPSE system becomes accessible to traders, data scientists, and management. It turns raw predictions into actionable insights and helps build trust in the model. In the next chapter, we will discuss **Data Visualization** in more depth, focusing on the principles and techniques for creating effective charts.

---

**End of Chapter 70**